In [20]:
import requests
from bs4 import BeautifulSoup
import bs4
import logging
from dataclasses import dataclass
import re
import cloudscraper
import time
import os
from tqdm import tqdm
import tqdm
import datetime
import itertools

In [22]:
requests.__version__

'2.28.1'

In [2]:
logs_path = 'logs'
scrape_log_file = os.path.join(logs_path, 'cian_scrapper.txt')

In [3]:
@dataclass
class Property:
    "Dataclass for keeping proeprty attributes"
    pass

In [4]:
class PropertyParser:
    "Scrapping info from every property"
    pass

In [5]:
# create decorator for logging

In [6]:
match_dict = {'1': 'room1=1',
'2': 'room2=1',
'3': 'room3=1',
'4': 'room4=1',
'5': 'room5=1',
'6': 'room6=1',
'studio': 'room9=1'}

In [8]:


class CianUrlsParser:
    def __init__(self, rooms):
        self.__MAX_COUNT_OF_PAGE = 54
        self.__MAX_COUNT_OF_ADDS = 28
        self.__MAX_COUNT_OF_PROPERTIES = self.__MAX_COUNT_OF_PAGE * self.__MAX_COUNT_OF_ADDS
        self.scrapper = cloudscraper.CloudScraper()
        self.rooms_url = match_dict[rooms]
        self.start_url = 'https://kemerovo.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=4795&' + self.rooms_url
        self.scrapper.headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML,[] like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
        self.logs_path = 'logs'
        self.scrape_log_file = os.path.join(logs_path, 'cian_scrapper.txt')  
        self.keep_data_path = 'data'
        
    def get_count_of_pages(self):
        page = self.scrapper.get(url = self.start_url)
        soup = BeautifulSoup(page.text, 'html.parser')
        text_count_page = soup.find('h5', {'class': '_93444fe79c--color_black_100--kPHhJ _93444fe79c--lineHeight_20px--tUURJ _93444fe79c--fontWeight_bold--ePDnv _93444fe79c--fontSize_14px--TCfeJ _93444fe79c--display_block--pDAEx _93444fe79c--text--g9xAG _93444fe79c--text_letterSpacing__normal--xbqP6'}).text
        count_of_adds = int(''.join(re.findall(r'\d+', text_count_page)))
        count_of_pages = count_of_adds // 50 + 1
        return count_of_pages

    def get_info_about_adds(self):
        count_of_pages = get_count_of_pages()
        ret = f"Count of pages: {count_of_pages}, potential count of add to scrape: {__MAX_COUNT_OF_PROPERTIES}"
    
    def write_to_file(self, values):
        with open(os.path.join(self.keep_data_path, 'rooms1_' + datetime.datetime.now().strftime("%m_%d_%Y-%H:%M:%S") + '.txt'), 'a+') as file:
            [file.write(value + '\n') for value in values]
    
    def scrape(self):
        count_of_pages = self.get_count_of_pages()
        if count_of_pages > self.__MAX_COUNT_OF_PAGE:
            print(f"Count of pages is more then {__MAX_COUNT_OF_PAGE}. Scrapping only {__MAX_COUNT_OF_PAGE} pages")
            count_of_pages = self.__MAX_COUNT_OF_PAGE
            
        print(f"Started scrapping. Count of pages: {count_of_pages}\n")
        all_hrefs = []
        for page in tqdm(range(1, count_of_pages + 1)):
            url = self.start_url + f'&p={page}'
            soup = BeautifulSoup(self.scrapper.get(url = url).text, 'html.parser')
            objects = soup.findAll('a', {'class': '_93444fe79c--media--9P6wN'})
            hrefs = [i.get('href') for i in objects]
            all_hrefs.append(hrefs)
            time.sleep(2)
        all_hrefs = list(itertools.chain(*all_hrefs))
        self.write_to_file(all_hrefs)
    
        

In [9]:
scrapper = CianUrlsParser('1')

In [10]:
scrapper.scrape()

  0%|          | 0/26 [00:00<?, ?it/s]

Started scrapping. Count of pages: 26



100%|██████████| 26/26 [01:50<00:00,  4.26s/it]
